HW Senator

Zihao Huang(ZH288)

In [1]:
import urllib.request
import pandas as pd
import sklearn
import sklearn.cluster
from itertools import combinations
import numpy as np
import itertools
from bokeh.plotting import output_file, show, figure
from bokeh.layouts import row
from bokeh.io import push_notebook, show, output_notebook
from ipywidgets import interact
output_notebook()
import time

Loading BokehJS ...

Download Data via urllib

In [2]:
with urllib.request.urlopen('https://voteview.com/static/data/out/members/HSall_members.csv') as url:
    data = url.read()
url_member = 'https://voteview.com/static/data/out/members/HSall_members.csv'
member = pd.read_csv(url_member)
party = member[['icpsr','party_code']]
party= party.drop_duplicates()
url_vote = 'https://voteview.com/static/data/out/votes/HSall_votes.csv'
vote = pd.read_csv(url_vote)
votetest=vote
votetest = votetest[votetest["chamber"]=="Senate"]
del votetest["chamber"]
del votetest["prob"]

In [3]:
votetest

,congress,rollnumber,icpsr,cast_code
7195,1,1,507,1
7196,1,1,1346,6
7197,1,1,1536,1
7198,1,1,2307,1
7199,1,1,2936,1
7200,1,1,2943,1
7201,1,1,3128,1
7202,1,1,3771,6
7203,1,1,3877,1
7204,1,1,4333,1


In [4]:
def getPlot(date):
    n115 = votetest[votetest['congress']==date]
    del n115['congress']
    del n115['rollnumber']
    result = n115.set_index("icpsr").groupby(level=0).apply(lambda n115:n115.reset_index(drop=True)).unstack().sort_index(axis=1,level=1)
    result=result.fillna(0.5)
    result = result.replace(2,1)
    result = result.replace(3,1)
    result = result.replace(4,0)
    result = result.replace(5,0)
    result = result.replace(6,0)
    result = result.replace(7,0.5)
    result = result.replace(8,0.5)
    result = result.replace(9,0.5)
    result['icpsr']=result.index
    result2 = pd.merge(result,party,on="icpsr",how="left")
    kmeans_model = sklearn.cluster.KMeans(n_clusters=2).fit(result2.iloc[:,1:-2])
    labels = kmeans_model.labels_
    result2['label']=[not x for x in labels]
    result2['party_code'] = result2['party_code'].replace(200,"R")
    result2['party_code'] = result2['party_code'].replace(100,"D")
    #PCA 2. Plot is really ugly
    #pca_2=sklearn.decomposition.PCA(2)
    #plot_columns = pca_2.fit_transform(result2.iloc[:,1:-3])
    #x=plot_columns[:,0]
    #y=plot_columns[:,1]
    #PCA 1
    pca_2=sklearn.decomposition.PCA(1)
    plot_columns = pca_2.fit_transform(result2.iloc[:,1:-3])
    ll = np.linspace(np.min(plot_columns[:,0]),np.max(plot_columns[:,0]),len(plot_columns[:,0]))
    x=plot_columns[:,0]
    y=ll
    #
    haha = pd.concat([result2,pd.DataFrame({'x':x,'y':y})],axis=1)
    count = 0
    demox = np.array([])
    demoy = np.array([])
    repubx = np.array([])
    repuby = np.array([])
    mixx = np.array([])
    mixy = np.array([])
    for l in itertools.combinations(haha.index.tolist(),2):
        #stats[l] = haha.loc[l[0],:], haha.loc[l[1],:])
        #print(haha.iloc[l[0],3:-3])
        #break
        count += 1
        arr = (np.array(haha.iloc[l[0],1:-5])-np.array(haha.iloc[l[1],1:-5]))
        #print(l)
        #print(np.count_nonzero(arr==0)/arr.shape[0])
        if np.count_nonzero(arr==0)/arr.shape[0] >= 0.50:

            xx = np.array([haha.iloc[l[0],-2],haha.iloc[l[1],-2]])
            yy = np.array([haha.iloc[l[0],-1],haha.iloc[l[1],-1]])
            if (haha.iloc[l[0],-4]=='D') & (haha.iloc[l[1],-4]=='D'):
                demox=np.concatenate((demox,xx))
                demoy=np.concatenate((demoy,yy))
            if (haha.iloc[l[0],-4]=='R') & (haha.iloc[l[1],-4]=='R'):
                repubx=np.concatenate((repubx,xx))
                repuby=np.concatenate((repuby,yy))
            if ((haha.iloc[l[0],-4]=='D') & (haha.iloc[l[1],-4]=='R')) | ((haha.iloc[l[0],-4]=='R') & (haha.iloc[l[1],-4]=='D')):
                mixx=np.concatenate((mixx,xx))
                mixy=np.concatenate((mixy,yy))
    #Put democrat to the right
    if demox.min()< repubx.min():
        demox = -demox
        repubx = -repubx
    p = figure(title="Congress"+str(date))
    p.triangle(x, y, legend="icpsr")
    #p.line(demox, demoy, line_color = "blue")
    #p.line(repubx, repuby, line_color = "red")
    #p.line(mixx, mixy, line_color = "black")
    p.multi_line(xs = [demox,repubx,mixx],ys = [demoy,repuby,mixy],color=['blue','red','green'],line_width=[0.05,0.05,0.2])  
    return p

Get plot for congress session 100 to 115(Recent 15 years)

In [5]:
lst = []
ti = 0 
endti = 0
totalti = 0
firsttime = True
aveagertime = 0
for i in range(90,116,1):
    ti = time.time()
   
    lst.append(getPlot(i))
    print(str(((i-90)/(116-90))*100)+"%")
    if firsttime:
        totalti = (116-90)*(time.time()-ti)
        averagetime = time.time()-ti
        firsttime = False
    endti = time.time()-ti
    print("Estimated Time Remaining: " + str(totalti))
    print("-------------------------------------------------")
    totalti = totalti - endti+(endti - averagetime)
    

E:\pythonana\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


0.0%
Estimated Time Remaining: 535.3465313911438
-------------------------------------------------
3.8461538461538463%
Estimated Time Remaining: 514.7562801837921
-------------------------------------------------
7.6923076923076925%
Estimated Time Remaining: 494.16602897644043
-------------------------------------------------
11.538461538461538%
Estimated Time Remaining: 473.57577776908875
-------------------------------------------------
15.384615384615385%
Estimated Time Remaining: 452.98552656173706
-------------------------------------------------
19.230769230769234%
Estimated Time Remaining: 432.3952753543854
-------------------------------------------------
23.076923076923077%
Estimated Time Remaining: 411.8050241470337
-------------------------------------------------
26.923076923076923%
Estimated Time Remaining: 391.214772939682
-------------------------------------------------
30.76923076923077%
Estimated Time Remaining: 370.6245217323303
--------------------------------------

In [6]:
def update(Congress_Number=112):
    show(lst[Congress_Number-90])
    push_notebook()

Blue line is connected when two democrats have the same votes for more than 50% of the rollcalls

Red line is connected when two republicans have the same votes for more than 50% of the rollcalls

Green line is connected when a democrat and a republican have the same votes for more than 50% of the rollcalls

In [7]:
interact(update, Congress_Number=(90,115))

<function __main__.update>

In [8]:
show(row(lst[0],lst[1],lst[2],lst[3],lst[4],lst[5],lst[6],lst[7],lst[8],lst[9],lst[10],lst[11],lst[12],lst[13],lst[14],lst[15]))

We can see a lot of green lines in congress 100. This is the time when democrats and republicans agree on a lot of the issues.

In recent congress 115, we can see much less green lines compared to congress 100. It shows that U.S senators are more divided than ever.